# Recognizing Emotions from Facial Expression Data

The data comes from a Kaggle competition:

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from logistic_regression import batch_gradient_ascent

In [ ]:
%load_ext line_profiler

In [ ]:
# Assuming the data has been downloaded to a CSV file
df = pd.read_csv('~/Downloads/fer2013.csv')
df

In [ ]:
emotion_labels=['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
# Put data into Numpy arrays
X = -1 * np.ones((len(df), 48, 48))
for i in range(len(df)):
    X[i,:,:] = np.array([int(s) for s in df['pixels'].loc[i].split()]).reshape(48,48)

In [ ]:
y = df['emotion'].values

In [ ]:
# display a random picture from the data set with label
row_index = np.random.choice(len(df))
plt.imshow(X[row_index, :, :], cmap='gray')
plt.title(emotion_labels[y[row_index]])

In [ ]:
# Reduce data set to classify happy vs. sad
is_happy_or_sad = (y==3) | (y==4)
X_happy_or_sad = X[is_happy_or_sad] / 255
y_happy_or_sad = y[is_happy_or_sad]
y_happy_or_sad[y_happy_or_sad==3] = 0
y_happy_or_sad[y_happy_or_sad==4] = 1

# # Reduce data set to angry vs. disgust
# X_angry = X[y==0] / 255
# X_disgust = np.repeat(X[y==1] / 255, 9, axis=0) # handle unblanaced data
# X_angry_or_disgust = np.vstack((X_angry, X_disgust))
# y_angry_or_disgust = np.array([0]*X_angry.shape[0] + [1]*X_disgust.shape[0])

In [ ]:
shuffled_indices = np.random.permutation(X_happy_or_sad.shape[0])
X_happy_or_sad = X_happy_or_sad[shuffled_indices,:,:]
y_happy_or_sad = y_happy_or_sad[shuffled_indices]

# shuffled_indices = np.random.permutation(X_angry_or_disgust.shape[0])
# X_angry_or_disgust = X_angry_or_disgust[shuffled_indices,:,:]
# y_angry_or_disgust = y_angry_or_disgust[shuffled_indices]

In [ ]:
# Train/test split
N = 2 * len(y_happy_or_sad) // 3  # number in train set
N_test = len(y_happy_or_sad) - N  # number in test set
X_train = X_happy_or_sad[:N,:,:].reshape((N, 48 * 48))
y_train = y_happy_or_sad[:N]
X_test = X_happy_or_sad[N:,:,:].reshape((N_test, 48 * 48))
y_test = y_happy_or_sad[N:]

# N = 2 * len(y_angry_or_disgust) // 3  # number in train set
# N_test = len(y_angry_or_disgust) - N  # number in test set
# X_train = X_angry_or_disgust[:N,:,:].reshape((N, 48 * 48))
# y_train = y_angry_or_disgust[:N]
# X_test = X_angry_or_disgust[N:,:,:].reshape((N_test, 48 * 48))
# y_test = y_angry_or_disgust[N:]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Profiling

Train the model with just 100 epochs for profiling purposes

In [ ]:
%lprun?

In [ ]:
%lprun -m logistic_regression batch_gradient_ascent(X_train, y_train, lr=1e-6, verbose=True, max_iters=100, step_size=20)

## Train the model

In [ ]:
t0 = datetime.datetime.now()
weights, train_costs, test_costs = batch_gradient_ascent(X_train, y_train, lr=1e-6, 
                                                         test_inputs=X_test, test_targets=y_test, 
                                                         verbose=True, max_iters=1000, step_size=20,
                                                         l1=True, l1_param=10)
print(f"{(datetime.datetime.now() - t0).total_seconds() / 60}")

In [ ]:
plt.plot(train_costs, label='Train Cost')
plt.plot(test_costs, label='Test Cost')
plt.legend()